In [ ]:
#Imports et utilitaires

In [ ]:
!pip install -q gcsfs shap optuna

import optuna
import pandas as pd, numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
from graphviz import Source
from IPython.display import SVG
%matplotlib inline

#dessine la graphe d'Elbow pour trouver le nombre idéal de clusters 
def findbestclusters(data,maxclusters=30,minclusters=2):
  Sum_of_squared_distances = []
  K = range(minclusters,maxclusters)
  for k in K:
      km = KMeans(n_clusters=k, random_state=49)
      km = km.fit(data)
      Sum_of_squared_distances.append(km.inertia_)
  plt.plot(K, Sum_of_squared_distances, 'bx-')
  plt.xlabel('k')
  plt.ylabel('Sum_of_squared_distances')
  plt.title('Elbow Method For Optimal k')
  plt.show()

  sil = []
  # dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
  for k in range(minclusters+1, maxclusters+1):
    kmeans = KMeans(n_clusters = k).fit(X)
    labels = kmeans.labels_
    sil.append({'k':k,'score':silhouette_score(X, labels, metric = 'euclidean')})
  sil=pd.DataFrame(sil)
  sil.plot(x='k',y='score')
  return sil[sil.score==sil.score.max()].iloc[0,0]

target='PERFORMANCE: resultat par uth familial'

In [ ]:
#Préparation de données

In [ ]:
##p

In [ ]:
!gsutil cp gs://referentiels2/viande.csv .
viande=pd.read_csv('viande.csv').dropna()
viande['ID_Dossier']=viande['ID_Dossier'].astype(int)
viande['DEP']=viande['DEP'].astype(int)
df=pd.read_csv('gs://referentiels2/laitratios.csv',sep=';')
df=df.merge(viande, how='left')
df['Produit_vente_animaux']=df['Produit_vente_animaux'].fillna(0)
df.to_csv('gs://referentiels2/laitratios1.csv',index=False)

In [ ]:
lait=pd.read_csv('gs://referentiels2/lait.csv',sep=';')
!gsutil cp gs://referentiels2/viande.csv .
viande=pd.read_csv('viande.csv').dropna()
viande['ID_Dossier']=viande['ID_Dossier'].astype(int)
viande['DEP']=viande['DEP'].astype(int)
lait=lait.merge(viande, how='left')
lait['PB_VEG']=lait['PRODUIT_BRUT']-lait['PRODUIT_LAIT_VALEUR_TOTALE']-lait['Produit_vente_animaux'].fillna(0)-lait['AUTRES_PRODUITS']
#PRODUIT BRUT – VENTES LAIT – VENTES ANIMAUX – AUTRES PRODUITS = PB_VEG.


In [ ]:
lait['PB_VEG_VLAIT']=lait['PB_VEG']/lait['PRODUIT_LAIT_QUANTITE']

In [ ]:
df=pd.read_csv('gs://referentiels2/laitratios1.csv')

In [ ]:
df=df.merge(lait[['DEP','ID_Dossier','Annee','PB_VEG_VLAIT']], how='left')

In [ ]:
df['PB_VEG_VLAIT']=df['PB_VEG_VLAIT'].fillna(0)

In [ ]:
lait['PRODUIT_BRUT_HA']=lait.PRODUIT_BRUT/lait['TOTAL_SAU']

In [ ]:
lait['VLAIT_HA']=lait['PRODUIT_LAIT_QUANTITE']/lait['TOTAL_SAU']

In [ ]:
df=df.merge(lait[['DEP','ID_Dossier','Annee','PRODUIT_BRUT_HA','VLAIT_HA']], how='left')

In [ ]:
df.to_csv('gs://referentiels2/laitratios1.csv',index=False)

In [ ]:
c=pd.read_csv('correctionnord.csv')

In [ ]:
c=c[c.DEP.isin(['59','62'])]

In [ ]:
c['DEP']=c['DEP'].astype(int)

In [ ]:
df=pd.read_csv('gs://referentiels2/laitratios1.csv')

In [ ]:
df=df.merge(c, how='left', on=['DEP','ID_Dossier','Annee'])

In [ ]:
df['VLAIT_PAR_UGB_VL']=np.where(df['VLAIT_PAR_UGB_VL_y'].isna(),df['VLAIT_PAR_UGB_VL_x'],df['VLAIT_PAR_UGB_VL_y'])

In [ ]:
df.drop(['VLAIT_PAR_UGB_VL_y','VLAIT_PAR_UGB_VL_x'],1,inplace=True)

In [ ]:
df.to_csv('gs://referentiels2/laitratios1.csv',index=False)

In [ ]:
##ajout du lait par sau et sfp

In [ ]:
lait=pd.read_csv('gs://referentiels2/lait.csv',sep=';')
complements=lait[['DEP','ID_Dossier','Annee','PRODUIT_LAIT_QUANTITE','TOTAL_SAU','SFP']].copy()
complements['lait_par_sau']=complements.PRODUIT_LAIT_QUANTITE/complements.TOTAL_SAU
complements['lait_par_sfp']=complements.PRODUIT_LAIT_QUANTITE/complements.SFP
complements=complements[['DEP','ID_Dossier','Annee','lait_par_sau','lait_par_sfp']].copy()
df=pd.read_csv('gs://referentiels2/laitratios1.csv')
df=df.merge(complements, how='left')
df.to_csv('gs://referentiels2/laitratios1.csv',index=False)

In [ ]:
#Nettoyage des valeurs extrêmes et remplacement des valeurs manquantes

In [ ]:
df=pd.read_csv('/content/BASE_ALL_PROJET_LAIT_VF_23032020.csv',sep=';')
df=df[df['SFP']>0]
df=df[df['VLAIT_PAR_VL']!='#DIV/0!']
df['VLAIT_PAR_VL']=df['VLAIT_PAR_VL'].astype(float)
df['POSITIONNEMENT: lait par ha de surface fourragère principale']=df['PRODUIT_LAIT_QUANTITE']/df['SFP']
df=df.rename(columns={'PRODUIT_BRUT_HA':'PRATIQUES D\'ELEVAGE: produit brut par hectare','PB_VEG_VLAIT':'PRATIQUES D\'ELEVAGE: ventes de végétaux','DEP':'ID: département','ID_Dossier':'ID: dossier','RESULTAT_PAR_UTH_FAM':'CIBLE: resultat par uth familial','UTH_FAMILIAUX':'DIMENSION: uth familiaux','TOTAL_ANNUITES_VLAIT':'DYNAMIQUE STRUCTURELLE: annuites','AUTRES_FRAIS_ELEVAGE_VLAIT':'PRATIQUES D\'ELEVAGE: autres frais','CHARGES_FINANCIERES_CT_VLAIT':'DYNAMIQUE STRUCTURELLE: charges financières court terme',
                  'CHARGES_FINANCIERES_LMT_VLAIT':'DYNAMIQUE STRUCTURELLE: charges financières long terme','HONORAIRES_VETO_PAR_VLAIT':'PRATIQUES D\'ELEVAGE: frais vétérinaires','SALAIRES_CHARGES_SOCIALES_VLAIT':'DYNAMIQUE STRUCTURELLE: salaires chargés',
                   'VLAIT_PAR_UTH_FAM':'DYNAMIQUE STRUCTURELLE: volume de lait par uth familial','SURFACE_MAIS_FOURRAGER':'DIMENSION: surface de maïs fourrager',
                   'TOTAL_CHARGES_STRUCTURE_VLAIT':'DYNAMIQUE STRUCTURELLE: charges de structure',
                   'UGB':'DIMENSION: taille du troupeau','VLAIT_PAR_VL':'POSITIONNEMENT: lait par vache laitière',
                   'Produit_vente_animaux':'PRATIQUES D\'ELEVAGE: vente d\'animaux','TAUX_SPECIALISATION':'PRATIQUES D\'ELEVAGE: taux de spécialisation','PRIX_UNITAIRE_LAIT':'INVARIANT: prix du lait','SFP':'DIMENSION: surface des prairies',
                   'CHARGES_ALIMENTS_PAR_VLAIT':'PRATIQUES D\'ELEVAGE: aliments','PRODUIT_BRUT_VLAIT':'PRATIQUES D\'ELEVAGE: produit brut',
                   'AMORTISSEMENTS_VLAIT':'DYNAMIQUE STRUCTURELLE: amortissements',
                   })

orders=['ID','INVARIANT','DIMENSION','DYNAMIQUE STRUCTURELLE','PRATIQUES D\'ELEVAGE','POSITIONNEMENT','CIBLE']
df=pd.concat([df.filter(regex=f'{order}:') for order in orders],1)
df=df.fillna(0)

#sélection des fermes avec 3 exercices minimum et moyenne des valeurs
indexcolumns=list(df.filter(regex='ID:').columns)
means=df.groupby(indexcolumns).mean()
counts=df.groupby(indexcolumns).count().iloc[:,0]
counts=counts[counts>2]
df=means[means.index.isin(counts.index)]
df=df[(df['POSITIONNEMENT: lait par ha de surface fourragère principale']<20000) & (df['POSITIONNEMENT: lait par vache laitière']>4000) & (df['POSITIONNEMENT: lait par vache laitière']<20000) & (df['DIMENSION: surface des prairies']<200) & (df['DIMENSION: taille du troupeau']<400) & (df['CIBLE: resultat par uth familial']>-20000)]

In [ ]:
df.shape #814 737

In [ ]:
#détermination des groupes

In [ ]:
n_clusters=findbestclusters(df.filter(regex='POSITIONNEMENT'),maxclusters=10,minclusters=1)

In [ ]:
n_clusters=5
km = KMeans(n_clusters=n_clusters, random_state=49)
y=km.fit_predict(df.filter(regex='POSITIONNEMENT'))
fig, ax = plt.subplots(figsize=(10,8)) 
sns.scatterplot(data=df, x='POSITIONNEMENT: lait par vache laitière',y='POSITIONNEMENT: lait par ha de surface fourragère principale',hue=y,ax=ax,palette="Set2",)

In [ ]:
dfclustered=df.copy()
dfclustered['cluster']=y
dfclustered.to_csv('gs://referentiels2/dfclustered.csv')

In [ ]:
dfclustered

In [ ]:
import plotly.express as px
fig = px.scatter_3d(dfclustered, x='POSITIONNEMENT: lait par vache laitière',y='POSITIONNEMENT: lait par ha de surface fourragère principale', z='CIBLE: resultat par uth familial',
              color='cluster')
fig.show()

In [ ]:
dfclustered.to_csv('dfclustered.csv')

In [ ]:
#détermination des critères de clusters

In [ ]:
dfclustered=pd.read_csv('gs://referentiels2/dfclustered.csv')

In [ ]:
sns.countplot(x='cluster',data=dfclustered, palette='Set2')

In [ ]:
for c in dfclustered.columns:
  fig, ax = plt.subplots(figsize=(4,4)) 
  sns.boxplot(x="cluster", y=c, data=dfclustered,
                   ax=ax, palette='Set2')

In [ ]:
dfclustered

In [ ]:
import plotly.graph_objects as go
values=dfclustered.groupby('cluster').mean()

values=pd.DataFrame(MinMaxScaler().fit_transform(values),columns=values.columns)

categories = values.columns
for index, row in values.iterrows():
  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
        r=list(row.values),
        theta=categories,
        fill='toself',
        name=str(index),
        line_color='rgb'+str(sns.color_palette("Set2",8)[index])
  ))
  fig.show()

In [ ]:
for cluster in dfclustered.cluster.unique():
  print(cluster)
  cluster0=dfclustered.copy()
  cluster0['cluster']=cluster0['cluster']==cluster
  cluster0=cluster0.drop(['ID: dossier','ID: département','POSITIONNEMENT: lait par ha de surface fourragère principale','POSITIONNEMENT: lait par vache laitière'],1)
  X=cluster0.drop('cluster',1)
  y=cluster0.cluster
  estimator = DecisionTreeClassifier(max_depth=3)
  estimator.fit(X, y)

  graph = Source(tree.export_graphviz(estimator, out_file=None
    , feature_names=X.columns
    , filled = True))
  display(SVG(graph.pipe(format='svg')))

In [ ]:
#analyse suite

In [ ]:
import lightgbm as lgb
import shap
dfclustered=pd.read_csv('gs://referentiels2/dfclustered.csv')
def bestandworst(cluster):
  c0=dfclustered[dfclustered.cluster==cluster].drop(['cluster','x','y'],1).set_index(['ID: département','ID: dossier','ID: année'])
  random_state = 7
  X = c0.drop('PERFORMANCE: resultat par uth familial',1)
  y=c0['PERFORMANCE: resultat par uth familial']
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
  model=lgb.LGBMRegressor(n_estimators=50000)
  model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20,verbose=False)
  explainer = shap.TreeExplainer(model)
  expected_value = explainer.expected_value
  features = X.loc[y.sort_values()[-20:].append(y.sort_values()[:20]).index]
  features_display = features
  shap_values = explainer.shap_values(features)#[1]
  shap_interaction_values = explainer.shap_interaction_values(features)
  shap.decision_plot(expected_value, shap_values, features_display)
  #shap.summary_plot(shap_values, X)

In [ ]:
dfclustered.to_csv('dfclustered.csv',index=False)

In [ ]:
bestandworst(0)

In [ ]:
bestandworst(0)

In [ ]:
bestandworst(1)

In [ ]:
bestandworst(2)

In [ ]:
bestandworst(3)

In [ ]:
bestandworst(4)

In [ ]:
from sklearn.neighbors import NearestNeighbors
X=dfclustered.set_index(['ID: département', 'ID: dossier', 'ID: année']).drop(['x','y'],1)
nbrs = NearestNeighbors(n_neighbors=100, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

In [ ]:
indices=pd.DataFrame(indices)

In [ ]:
i=[]
for index,row in indices.iterrows():
  for j in row[1:]:
    i.append({'index':index,'neighbor':j})

In [ ]:
i=pd.DataFrame(i)

In [ ]:
neighbors=i.merge(dfclustered[['PERFORMANCE: resultat par uth familial']],left_on='neighbor', right_index=True)

In [ ]:
best=neighbors.merge(dfclustered, left_on='index', right_index=True).merge(dfclustered, left_on='neighbor', right_index=True, suffixes=('', '_target'))

In [ ]:
best=best[best["EXPLOITATION: uth familiaux"]==best["EXPLOITATION: uth familiaux_target"]]
best=best[(np.abs(best['EXPLOITATION: surface des prairies']-best['EXPLOITATION: surface des prairies_target'])/best['EXPLOITATION: surface des prairies_target'])<0.1]
best=best[(np.abs(best['CHARGES: amortissements']-best['CHARGES: amortissements_target'])/best['CHARGES: amortissements_target'])<0.2]

In [ ]:
best

In [ ]:
best=best.groupby('index')['PERFORMANCE: resultat par uth familial_x'].max().reset_index().merge(best).drop_duplicates(subset='index')

In [ ]:
best=best.drop('PERFORMANCE: resultat par uth familial_x',1).rename(columns={'PERFORMANCE: resultat par uth familial':'PERFORMANCE: resultat par uth familial_target','PERFORMANCE: resultat par uth familial_y':'PERFORMANCE: resultat par uth familial'})

In [ ]:
for c in best.filter(regex='target').columns:
  c_init=c.replace('_target','')
  best[c_init+'_delta']=best[c]-best[c_init]
  best[c_init+'_delta_perc']=(best[c]-best[c_init])/best[c_init]

In [ ]:
best=best[best['PERFORMANCE: resultat par uth familial_delta']>0]

In [ ]:
best['PERFORMANCE: resultat par uth familial_delta_perc'].describe()

In [ ]:
best[best['cluster']==0].cluster.value_counts().sort_index()

In [ ]:
best[best['cluster']==0].cluster_target.value_counts().sort_index()

In [ ]:
best.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10,8)) 
sns.scatterplot(data=best, x='x_target',y='y_target',hue='PERFORMANCE: resultat par uth familial_delta',ax=ax,palette="Reds",)

In [ ]:
best.groupby('cluster').mean().filter(regex='delta$')[['EXPLOITATION: uth familiaux_delta',
       'EXPLOITATION: taux de spécialisation_delta',
       'EXPLOITATION: surface des prairies_delta',
       'EXPLOITATION: surface de maïs fourrager_delta',
       'ELEVAGE: autres frais_delta', 'ELEVAGE: frais vétérinaires_delta',
       'ELEVAGE: aliments_delta', 'ELEVAGE: taille du troupeau_delta',
       'ELEVAGE: vente d\'animaux_delta', 'CONTEXTE: prix du lait_delta',
       'CONTEXTE: aides laitières_delta', 'CHARGES: annuites_delta',
       'CHARGES: amortissements_delta',
       'CHARGES: charges financières court terme_delta',
       'CHARGES: charges financières long terme_delta',
       'CHARGES: salaires chargés_delta',
       'CHARGES: charges de structure_delta',
       'PERFORMANCE: resultat par uth familial_delta',
       'PERFORMANCE: produit brut_delta',
       'PERFORMANCE: volume de lait par uth familial_delta',
       'PERFORMANCE: lait par vache laitière_delta']]

In [ ]:
best['PERFORMANCE: resultat par uth familial'].describe()

In [ ]:
best['PERFORMANCE: resultat par uth familial_target'].describe()

In [ ]:
resume=best.groupby('cluster').mean().filter(regex='perc$')

In [ ]:
resume[np.abs(resume)>0.05]

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
vals=resume[np.abs(resume)>0.05].T.fillna(0).replace(np.inf,0).iloc[3:-3,:] 
sns.heatmap(vals.drop(['CHARGES: annuites_delta_perc','PERFORMANCE: resultat par uth familial_delta_perc']), cmap='Blues',ax=ax)

In [ ]:
dfshort[target]=np.where(
    dfscaled[target]<dfscaled[target].mean()-dfscaled[target].std(),-2,
    np.where(dfscaled[target]<dfscaled[target].mean(),-1,
             np.where(dfscaled[target]<dfscaled[target].mean()+dfscaled[target].std(),1,2)))
fig, ax = plt.subplots(figsize=(10,8)) 
sns.scatterplot(data=dfshort, x='x',y='y',hue=target,ax=ax,palette="Reds",)

In [ ]:
import xgboost,shap
def importance(X,y):
  model = xgboost.train({"learning_rate": 0.01,"objective": 'multi:softmax','num_class':n_clusters}, xgboost.DMatrix(X, label=y), 100)
  explainer = shap.TreeExplainer(model)
  shap_values = explainer.shap_values(X)
  shap.initjs()
  shap.summary_plot(shap_values, X)

In [ ]:
importance(df.drop(target,1),y)

In [ ]:
centers = pd.DataFrame(km.cluster_centers_, columns=dfscaled.drop(target,1).columns)

In [ ]:
dfscaledwithcluster=dfscaled
dfscaledwithcluster['cluster']=y

In [ ]:
dfscaledwithcluster=dfscaledwithcluster.groupby('cluster').mean()

In [ ]:
fig, ax = plt.subplots(figsize=(10,4)) 
sns.heatmap(dfscaledwithcluster[[target,'PERFORMANCE: produit brut','CHARGES: salaires chargés','PERFORMANCE: lait par gros bovin','ELEVAGE: taille du troupeau','CHARGES: annuites','CHARGES: charges de structure','EXPLOITATION: surface des prairies']],ax=ax,cmap="Blues")

In [ ]:
X=df.drop([target,'ID: dossier','ID: département'],1)
y=df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
model=lgb.LGBMRegressor()
model.fit(X_train,y_train, eval_set=(X_test,y_test), verbose=False)
print(mean_absolute_error(y_test, model.predict(X_test)))
lgb.plot_importance(model, max_num_features=40, figsize=(12,12))

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X)
shap.initjs()
shap.summary_plot(shap_values, X)

In [ ]:
#optimisations

In [ ]:
dfclustered=pd.read_csv('gs://referentiels2/dfclustered.csv')

In [ ]:
#passer les colonnes de positionnements en pratiques d'élevages
dfclustered=dfclustered.rename(columns={'POSITIONNEMENT: lait par vache laitière':'PRATIQUES D\'ELEVAGE: lait par vache laitière',
       'POSITIONNEMENT: lait par ha de surface fourragère principale':'PRATIQUES D\'ELEVAGE: lait par ha de surface fourragère principale'})
#création du modèle de prédiction de cible
import lightgbm as lgb
random_state = 7
X = dfclustered.drop(['CIBLE: resultat par uth familial','cluster','ID: département','ID: dossier'],1)
y=dfclustered['CIBLE: resultat par uth familial']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
model=lgb.LGBMRegressor(n_estimators=50000)
model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20,verbose=False)
model.score(X_test,y_test)
#création du modèle de cluster
X = dfclustered[['PRATIQUES D\'ELEVAGE: lait par vache laitière','PRATIQUES D\'ELEVAGE: lait par ha de surface fourragère principale']]
y=dfclustered['cluster']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)
modelcluster=lgb.LGBMClassifier(n_estimators=50000)
modelcluster.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20,verbose=False)
modelcluster.score(X_test,y_test)

In [ ]:
import joblib
joblib.dump(model,'model')
joblib.dump(modelcluster,'modelcluster')

In [ ]:
!gsutil cp model gs://referentiels2/
!gsutil cp modelcluster gs://referentiels2/


In [ ]:
!gsutil -m rm -r gs://referentiels2/cerfrance*.csv

In [ ]:
dfclustered[[]].to_csv('gs://findstorage/cerfrance/simulation7.csv',index=True)

In [ ]:
!gsutil -m cp -r gs://referentiels2/cerfrance*.csv .

In [ ]:
import os
files=[f for f in os.listdir() if '.csv' in f]

In [ ]:
simulation=[]
for f in files:
  try:
    simulation.append(pd.read_csv(f) )
  except:
    print('error for '+f)

In [ ]:
simulation=pd.concat(simulation)

In [ ]:
simulation=simulation.dropna()

In [ ]:
simulation['index']=simulation['index'].astype(int)

In [ ]:
simulation.head()

In [ ]:
simulationresults=dfclustered.merge(simulation, left_index=True, right_on=['index']).drop('index',1)

In [ ]:
effort=np.abs((simulationresults.filter(regex='_y').drop(['cluster_y','CIBLE: resultat par uth familial_y'],1).values-simulationresults.filter(regex='_x').drop(['cluster_x','CIBLE: resultat par uth familial_x'],1).values)/simulationresults.filter(regex='_x').drop(['cluster_x','CIBLE: resultat par uth familial_x'],1).values)

In [ ]:
effort=pd.DataFrame(effort).replace(np.inf,0).mean(1)

In [ ]:
simulationresults['effort']=effort

In [ ]:
simulationresults['gain']=simulationresults['CIBLE: resultat par uth familial_y']-simulationresults['CIBLE: resultat par uth familial_x']
simulationresults['gainrelatif']=(simulationresults['CIBLE: resultat par uth familial_y']-simulationresults['CIBLE: resultat par uth familial_x'])/simulationresults['CIBLE: resultat par uth familial_x']

In [ ]:
simulationresults['interessant']=(simulationresults['gain']>0) & (simulationresults['effort']<1)

In [ ]:
final=simulationresults[simulationresults['interessant']==True].groupby(['cluster_x','cluster_y','colonnes'])['effort','gain'].mean()

In [ ]:
final['rentabilite']=final.gain/final.effort

In [ ]:
final.sort_values(by='rentabilite',ascending=False)

In [ ]:
simulationresults.to_csv('gs://referentiels2/simulationresults.csv',index=False)

In [ ]:
simulationresults=pd.read_csv('gs://referentiels2/simulationresults.csv')

In [ ]:
simulationresults.head(n=10)

In [ ]:
simulationresults['gain'].describe()

In [ ]:
simulationresults.groupby(['cluster_x','colonnes']).gain.median()

In [ ]:
simulationresults[simulationresults.cluster_x==simulationresults.cluster_y].groupby(['cluster_x','colonnes']).gainrelatif.median()

In [ ]:
simulationresults[simulationresults.cluster_x!=simulationresults.cluster_y].groupby(['cluster_x','cluster_y']).gainrelatif.median()

In [ ]:
simulationresults.colonnes.value_counts()

In [ ]:
import plotly.graph_objects as go
colonnes='DIMENSION'#'DYNAMIQUE_STRUCTURELLE'
values=simulationresults[simulationresults['colonnes']==colonnes].drop(['ID: dossier','ID: département'],1).filter(regex='_y').groupby('cluster_y').mean()

values=pd.DataFrame(MinMaxScaler().fit_transform(values),columns=values.columns)

categories = values.columns
for index, row in values.iterrows():
  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
        r=list(row.values),
        theta=categories,
        fill='toself',
        name=str(index),
        line_color='rgb'+str(sns.color_palette("Set2",8)[index])
  ))
  fig.show()

In [ ]:
colonnes='DYNAMIQUE STRUCTURELLE'
values=simulationresults[simulationresults['colonnes']==colonnes].drop(['ID: dossier','ID: département'],1).filter(regex='_y').groupby('cluster_y').mean()

values=pd.DataFrame(MinMaxScaler().fit_transform(values),columns=values.columns)

categories = values.columns
for index, row in values.iterrows():
  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
        r=list(row.values),
        theta=categories,
        fill='toself',
        name=str(index),
        line_color='rgb'+str(sns.color_palette("Set2",8)[index])
  ))
  fig.show()

In [ ]:
import plotly.graph_objects as go
colonnes='PRATIQUES D\'ELEVAGE'
values=simulationresults[simulationresults['colonnes']==colonnes].drop(['ID: dossier','ID: département'],1).filter(regex='_y').groupby('cluster_y').mean()

values=pd.DataFrame(MinMaxScaler().fit_transform(values),columns=values.columns)

categories = values.columns
for index, row in values.iterrows():
  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
        r=list(row.values),
        theta=categories,
        fill='toself',
        name=str(index),
        line_color='rgb'+str(sns.color_palette("Set2",8)[index])
  ))
  fig.show()

In [ ]:
colonnes='PRATIQUES D\'ELEVAGE|DYNAMIQUE STRUCTURELLE|DIMENSION'
values=simulationresults[(simulationresults.cluster_x==simulationresults.cluster_y) & (simulationresults['colonnes']==colonnes)].drop(['ID: dossier','ID: département'],1).filter(regex='_y').groupby('cluster_y').mean()

values=pd.DataFrame(MinMaxScaler().fit_transform(values),columns=values.columns)

categories = values.columns
for index, row in values.iterrows():
  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
        r=list(row.values),
        theta=categories,
        fill='toself',
        name=str(index),
        line_color='rgb'+str(sns.color_palette("Set2",8)[index])
  ))
  fig.show()

In [ ]:
simulationresults[simulationresults.cluster_x!=simulationresults.cluster_y].cluster_y.value_counts()

In [ ]:
colonnes='PRATIQUES D\'ELEVAGE|DYNAMIQUE STRUCTURELLE|DIMENSION'
values=simulationresults[(simulationresults.cluster_x!=simulationresults.cluster_y) & (simulationresults['colonnes']==colonnes)].drop(['ID: dossier','ID: département'],1).filter(regex='_y').groupby('cluster_y').mean()
values=pd.DataFrame(MinMaxScaler().fit_transform(values),columns=values.columns)

categories = values.columns
for index, row in values.iterrows():
  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
        r=list(row.values),
        theta=categories,
        fill='toself',
        name=str(index),
        line_color='rgb'+str(sns.color_palette("Set2",8)[index])
  ))
  fig.show()

In [ ]:
colonnes='PRATIQUES D\'ELEVAGE|DYNAMIQUE STRUCTURELLE|DIMENSION'
values=simulationresults[(simulationresults.cluster_y==1) & (simulationresults.cluster_x!=simulationresults.cluster_y) & (simulationresults['colonnes']==colonnes)].drop(['ID: dossier','ID: département'],1).filter(regex='_y').groupby('cluster_y').mean()

values=pd.DataFrame(MinMaxScaler().fit_transform(values),columns=values.columns)

categories = values.columns
for index, row in values.iterrows():
  fig = go.Figure()
  fig.add_trace(go.Scatterpolar(
        r=list(row.values),
        theta=categories,
        fill='toself',
        name=str(index),
        line_color='rgb'+str(sns.color_palette("Set2",8)[index])
  ))
  fig.show()

In [ ]:
delta=simulationresults.filter(regex='_y').values-simulationresults.filter(regex='_x').values
deltarelatif=(simulationresults.filter(regex='_y').values-simulationresults.filter(regex='_x').values)/simulationresults.filter(regex='_x').values

In [ ]:
delta.shape

In [ ]:
deltarelatif.shape

In [ ]:
simulationresults.shape

In [ ]:
delta=pd.DataFrame(delta, columns=[c.replace('_y','_delta') for c in simulationresults.filter(regex='_y').columns],index=simulationresults.index)
deltarelatif=pd.DataFrame(deltarelatif, columns=[c.replace('_y','_deltarelatif') for c in simulationresults.filter(regex='_y').columns],index=simulationresults.index)

In [ ]:
simulationresults=pd.concat([simulationresults,delta,deltarelatif],1).drop(['CIBLE: resultat par uth familial_deltarelatif','cluster_deltarelatif'],1)

In [ ]:
simulationresults

In [ ]:
simulationresults.to_csv('gs://referentiels2/simulationresults.csv',index=False, sep=';')

In [ ]:
simulationresults.to_csv('simulationresults.csv',index=False, sep=';')

In [ ]:
import pandas as pd, numpy as np

In [ ]:
!pip install -q gcsfs

In [ ]:
s=pd.read_csv('gs://referentiels2/simulationresults.csv', sep=';')

In [ ]:
s['Objectifs en productivité L / UTH Fam']=np.where((s.gain>0) & (s['DYNAMIQUE STRUCTURELLE: volume de lait par uth familial_delta']<-30), 'Réduire le volume lait /UTH de '+s['DYNAMIQUE STRUCTURELLE: volume de lait par uth familial_delta'].astype(int).astype(str)+' l',
  np.where((s.gain>0) & (s['DYNAMIQUE STRUCTURELLE: volume de lait par uth familial_delta']>30), 'Augmenter le volume lait /UTH de '+s['DYNAMIQUE STRUCTURELLE: volume de lait par uth familial_delta'].astype(int).astype(str)+' l',np.NaN))
limite=200
objectif='Productivité L / ha SFP'
colonne="PRATIQUES D'ELEVAGE: lait par ha de surface fourragère principale_delta"
s['Objectifs en '+objectif]=np.where((s.gain>0) & (s[colonne]<-limite), 'Réduire la '+objectif+' de '+s[colonne].astype(int).astype(str)+' l',
  np.where((s.gain>0) & (s[colonne]>limite), 'Augmenter la '+objectif+' de '+s[colonne].astype(int).astype(str)+' l',''))
limite = 100
objectif='Productivité L / VL'
colonne="PRATIQUES D'ELEVAGE: lait par vache laitière_delta"
s['Objectifs en '+objectif]=np.where((s.gain>0) & (s[colonne]<-limite), 'Réduire la '+objectif+' de '+s[colonne].astype(int).astype(str)+' l',
  np.where((s.gain>0) & (s[colonne]>limite), 'Augmenter la '+objectif+' de '+s[colonne].astype(int).astype(str)+' l',''))
objectif='INVESTIR'
colonne='DYNAMIQUE STRUCTURELLE: amortissements_delta'
s[objectif]=np.where((s.gain>0) & (s[colonne]<=-10), 'Réduire les investissements',
  np.where((s.gain>0) & (s[colonne]<=-5), 'Freiner les investissements',
           np.where((s.gain>0) & (s[colonne]>=10), 'Investir',
                    np.where((s.gain>0) & (s[colonne]>=5), 'Investir un peu plus',''))))
objectif="MAIN D'ŒUVRE SALARIEE"
limite = 5
colonne="DYNAMIQUE STRUCTURELLE: salaires chargés_delta"
s[objectif]=np.where((s.gain>0) & (s[colonne]<-limite), 'Réduire la MO salariée',
  np.where((s.gain>0) & (s[colonne]>limite), 'Embaucher',''))
objectif="COUT ALIMENTAIRE"
limite = 0
colonne="PRATIQUES D'ELEVAGE: aliments_delta"
s[objectif]=np.where((s.gain>0) & (s[colonne]<-limite), 'Baisser coût alimentaire',
  np.where((s.gain>0) & (s[colonne]>limite), 'Augmenter coût alimentaire',''))
objectif="Surf prairies"
limite = 0
colonne="DIMENSION: surface des prairies_delta"
s[objectif]=np.where((s.gain>0) & (s[colonne]<-limite), 'Baisser',
  np.where((s.gain>0) & (s[colonne]>limite), 'Augmenter',''))
objectif="Surf maïs"
limite = 0
colonne="DIMENSION: surface de maïs fourrager_delta"
s[objectif]=np.where((s.gain>0) & (s[colonne]<-limite), 'Baisser',
  np.where((s.gain>0) & (s[colonne]>limite), 'Augmenter',''))
s.columns=[c.replace(': ','_').replace(' ','_').replace("'",'_').replace('é','e').replace('ï','i').replace('è','e').replace('/','par').replace('Œ','oe') for c in s.columns]

In [ ]:
s.to_gbq('cerfrance.lait', project_id='tinbiwebsite', if_exists='replace')

In [ ]:
s.query('ID_dossier=="2090"')